In [ ]:
import pandas as pd
import xgboost as xgb
import PySimpleGUI as sg
from PIL import Image

# Load the dataset
df = pd.read_excel('D:\\1.Study\\1. Thesis\\1. Research-8 (Pile ML)\\Input\\Pile Shear ML final.xlsx')

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Train the model
lr = xgb.XGBRegressor(n_estimators=150, learning_rate=0.08, subsample=0.25,
                      colsample_bytree=0.7, max_depth=16, min_child_weight=1)
lr.fit(X, y)

# PySimpleGUI layout
sg.theme('DefaultNoMoreNagging')

layout = [
    [
        sg.Column(layout=[
            [
                sg.Frame(layout=[                  
                    [sg.Text('teff/D', size=(40, 1),font=('Times New Roman', 14)), sg.InputText(key='-D-', size=(20, 2))],
                    [sg.Text('a/d', size=(40, 1),font=('Times New Roman', 14)), sg.InputText(key='-tee/d-', size=(20, 2))],
                    [sg.Text('Prestressing strands index', size=(40, 1),font=('Times New Roman', 14)), sg.InputText(key='-rho_lp_f_yp-', size=(20, 2)),sg.Text('MPa',font=('Times New Roman', 14))],
                    [sg.Text('Non-prestressed strands index', size=(40, 1),font=('Times New Roman', 14)), sg.InputText(key='-rho_lnp_f_ynp-', size=(20, 2)),sg.Text('MPa',font=('Times New Roman', 14))],
                    [sg.Text('Effective shear reinforcement index', size=(40, 1),font=('Times New Roman', 14)), sg.InputText(key='-rho_s_f_ys-', size=(20, 2)),sg.Text('MPa',font=('Times New Roman', 14))],
                    [sg.Text('Pile concrete strength', size=(40, 1),font=('Times New Roman', 14)), sg.InputText(key='-fc-', size=(20, 2)),sg.Text('MPa',font=('Times New Roman', 14))],
                    [sg.Text('Total effective stress', size=(40, 1),font=('Times New Roman', 14)), sg.InputText(key='-f_ce-', size=(20, 2)),sg.Text('MPa',font=('Times New Roman', 14))],
             [sg.Frame(layout=[[sg.Text('Pile Shear Strength', size=(39, 1), font=('Times New Roman', 14)), sg.InputText(key='-OP-', size=(20, 2)), sg.Text('MPa',font=('Times New Roman', 14))]], title='Output',font=('Times New Roman', 16))],
                    [sg.Button('Predict', size=(10, 1), font=('Times New Roman', 14)), sg.Button('Cancel', size=(10, 1), font=('Times New Roman', 14))]
                ], title='Input parameters', font=('Times New Roman', 16))  # Added title here: Button size 14 and Times New Roman font
            ]
        ])
    ]
]
# Load images
image_path_1 = r"C:\Users\tanvi\OneDrive - Military Institute of Science and Technology (MIST)\1.Study\ML Research\ML Pile\Results\Shap_Summary_Plot_Pile.jpg"
image_path_2 = r"C:\Users\tanvi\OneDrive - Military Institute of Science and Technology (MIST)\1.Study\ML Research\ML Pile\Results\Shap_Mean_Plot_Pile.jpg"

# Open the images
imag1 = Image.open(image_path_1)
imag2 = Image.open(image_path_2)

# Get the original size of the images
width, height = imag1.size
width2, height2 = imag2.size

# Define the scale factor
scale_factor = .12

# Resize the images
img1 = imag1.resize((int(width * scale_factor), int(height * scale_factor)))
img2 = imag2.resize((int(width2 * scale_factor), int(height2 * scale_factor)))

# Save the resized images
img1.save('image11.png')
img2.save('image22.png')

# To add figures in two columns
fig1 = sg.Image(filename='image11.png', key='-fig1-', size=(width * scale_factor, height * scale_factor))
fig2 = sg.Image(filename='image22.png', key='-fig2-', size=(width2 * scale_factor, height2 * scale_factor))

# To layout the figures and descriptions in two columns
layout += [[sg.Text(' ')],
           [sg.Text('Summary plot and global importance of the input features',font=('Times New Roman', 14))],
           [fig1, fig2]]

window = sg.Window('Shear Strength Prediction', layout)

while True:
    event, values = window.read()
    if event == sg.WINDOW_CLOSED or event == 'Exit':
        break
    if event == 'Predict':
        try:
            input_data = {
                "teff/D": float(values['-D-']),
                "a/d": float(values['-tee/d-']),
                "Prestressing strands (MPa)": float(values['-rho_lp_f_yp-']),
                "Non-prestressed strands index (MPa)": float(values['-rho_lnp_f_ynp-']),
                "Effective shear reinforcement index (Maa)": float(values['-rho_s_f_ys-']),
                "Pile concrete strength (MPa)": float(values['-fc-']),
                "Total effective stress (MPa)": float(values['-f_ce-'])
            }
            input_df = pd.DataFrame([input_data])
            prediction = lr.predict(input_df)
#            window['-OUTPUT-'].update(f'Shear Strength Prediction: {prediction[0]:.2f} MPa')
            window['-OP-'].update(f'{prediction[0]:.2f}')
        except ValueError:
            window['-OUTPUT-'].update('Please enter valid numeric values')
        except Exception as e:
            window['-OUTPUT-'].update(f'Error occurred: {e}')

window.close()
